In [162]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm

In [207]:
#data_prefix = "ling_"
course = pd.read_csv(data_prefix + "data/course.csv") 
course_element = pd.read_csv(data_prefix + "data/course_element.csv") 
course_module = pd.read_csv(data_prefix + "data/course_module.csv") 
sol_log = pd.read_csv(data_prefix + "data/solution_log.csv") 
u_course_progress = pd.read_csv(data_prefix + "data/user_course_progress.csv") 
u_element_progress = pd.read_csv(data_prefix + "data/user_element_progress.csv") 
u_module_progress = pd.read_csv(data_prefix + "data/user_module_progress.csv")

Мои фичи: 
course_done - получен ли сертификат 1 уровня
progress_current_ratio - доля полученных баллов за курс
progress_current - сколько всего данных полученно 
total_courses_done - сколько всего закрыл курсов ученик
total_tries_overall - сколько попыток сдачи сделал на всех курсах 
tries_count - сколько попыток сделал ученик на данном курсе


In [213]:
course_to_max_points

{10: 646.0, 88: 645.0, 248: 653.0}

In [209]:
## Посчитаем, сколько в курсе можно заработать максимум баллов
course_to_max_points = dict(course_module.groupby("course_id").progress_max.sum())


## Подсчитаем, закрыл ли пользователь курс на сертификат первого уровня

d = dict(course_module[~course_module.is_advanced & course_module.level == 1].groupby(by="course_id").id.unique().apply(len))
u_course_progress["course_done"] = u_course_progress.course_id.map(d) <= u_course_progress.modules_achieved


## Подсчитаем по пользователю, сколько баллов он заработал

u_course_progress = u_course_progress.merge(u_element_progress.groupby(['user_id', 'course_id']).\
                                            progress_current.sum(), on=['user_id', 'course_id'])

## Найдем долю баллов которую получил пользователь от максимального числа баллов

u_course_progress["progress_current_ratio"] = u_course_progress.progress_current / u_course_progress.course_id.map(course_to_max_points)


## Найдем, сколько всего ученик закрыл курсов
u_course_progress["total_courses_done"] = u_course_progress.user_id.map(dict(u_course_progress.groupby("user_id").course_done.sum()))

u_course_progress.to_csv("user_course_progress_done.csv", index=None)




## Найдем, сколько всего попыток сдачи задач, сделал ученик. total_tries_overall - считая все курсы, 
## total_tries - учитывая только данный курс
u_course_progress["total_tries_overall"] = u_course_progress.user_id.map(dict(u_element_progress.groupby("user_id").tries_count.sum()))
u_course_progress = u_course_progress.merge( u_element_progress.groupby(["user_id", "course_id"]) \
                        .tries_count.sum(), on=['user_id', 'course_id'], how = "left")


In [212]:
course_module

,id,course_id,percent_to_pass,is_advanced,progress_max,steps_max,tasks_max,type,level
0,188,10,70.0,False,28.0,22,13,ordinary,1
1,201,10,70.0,False,130.0,46,38,ordinary,1
2,212,10,70.0,False,34.0,30,20,ordinary,1
3,186,10,70.0,False,26.0,22,16,ordinary,1
4,206,10,70.0,False,53.0,48,35,ordinary,1
5,203,10,70.0,False,35.0,25,14,ordinary,1
6,165,10,70.0,False,14.0,20,13,ordinary,1
7,200,10,70.0,False,22.0,25,17,ordinary,1
8,213,10,70.0,False,113.0,59,44,ordinary,1
9,215,10,70.0,False,50.0,42,30,ordinary,1


In [211]:
u_course_progress

,id,user_id,course_id,modules_achieved,advanced_modules_done,time_created,time_updated,modules_closed,modules_done,course_done,progress_current,progress_current_ratio,total_courses_done,total_tries_overall,tries_count
0,55444,100120050111539982,10,1,0,2019-11-07 14:24:08.490237,2019-11-08 13:02:35.117082,0,0,False,44.0,0.068111,0,407,253
1,45608,100120030110035096,10,13,0,2019-09-08 13:28:42.734497,2019-09-29 13:36:50.745967,13,13,True,646.0,1.000000,1,1240,846
2,50244,100119740110754751,10,0,0,2019-09-28 21:33:31.166358,NaN,0,0,False,0.0,0.000000,0,0,0
3,51879,100119930110760740,10,0,0,2019-10-14 05:30:27.101024,NaN,0,0,False,0.0,0.000000,0,0,0
4,51644,100120030110759694,10,0,0,2019-10-09 18:16:45.162979,NaN,0,0,False,0.0,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18863,234384,100120050111089766,248,0,0,2020-10-30 14:09:17.671742,NaN,0,0,False,0.0,0.000000,0,0,0
18864,217638,100120060111602010,248,0,0,2020-10-04 07:44:52.342324,NaN,0,0,False,0.0,0.000000,0,0,0
18865,225941,100120090111179931,248,0,0,2020-10-15 19:20:35.765176,NaN,0,0,False,0.0,0.000000,0,0,0
18866,224736,100119930111614862,248,0,0,2020-10-13 21:05:28.521268,NaN,0,0,False,0.0,0.000000,0,0,0


In [210]:
u_course_progress

,id,user_id,course_id,modules_achieved,advanced_modules_done,time_created,time_updated,modules_closed,modules_done,course_done,progress_current,progress_current_ratio,total_courses_done,total_tries_overall,tries_count
0,55444,100120050111539982,10,1,0,2019-11-07 14:24:08.490237,2019-11-08 13:02:35.117082,0,0,False,44.0,0.068111,0,407,253
1,45608,100120030110035096,10,13,0,2019-09-08 13:28:42.734497,2019-09-29 13:36:50.745967,13,13,True,646.0,1.000000,1,1240,846
2,50244,100119740110754751,10,0,0,2019-09-28 21:33:31.166358,NaN,0,0,False,0.0,0.000000,0,0,0
3,51879,100119930110760740,10,0,0,2019-10-14 05:30:27.101024,NaN,0,0,False,0.0,0.000000,0,0,0
4,51644,100120030110759694,10,0,0,2019-10-09 18:16:45.162979,NaN,0,0,False,0.0,0.000000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18863,234384,100120050111089766,248,0,0,2020-10-30 14:09:17.671742,NaN,0,0,False,0.0,0.000000,0,0,0
18864,217638,100120060111602010,248,0,0,2020-10-04 07:44:52.342324,NaN,0,0,False,0.0,0.000000,0,0,0
18865,225941,100120090111179931,248,0,0,2020-10-15 19:20:35.765176,NaN,0,0,False,0.0,0.000000,0,0,0
18866,224736,100119930111614862,248,0,0,2020-10-13 21:05:28.521268,NaN,0,0,False,0.0,0.000000,0,0,0


In [205]:
u_element_progress.groupby(["user_id", "course_id"]) \
                        .tries_count.sum()

user_id             course_id
100119700110000277  88            18
100119700110026069  10            14
                    88             0
100119700110044134  10             6
100119700110045876  10           129
                                ... 
100120190110504232  248            0
100120190110520171  10            37
                    88            17
100120190110636130  10             1
100120190110714343  88           611
Name: tries_count, Length: 18868, dtype: int64

In [206]:
u_course_progress[u_course_progress.user_id == 100119700110000277   ]

,id,user_id,course_id,modules_achieved,advanced_modules_done,time_created,time_updated,modules_closed,modules_done,course_done,progress_current,progress_current_ratio,total_courses_done,total_tries_overall,tries_count
14573,101676,100119700110000277,88,0,0,2020-03-23 12:37:10.259851,2020-04-16 12:42:36.850332,0,0,False,7.0,0.010853,0,18,18


In [172]:
u_course_progress["total_courses_done"].value_counts()

0    17759
1      838
2      241
3       30
Name: total_courses_done, dtype: int64

In [130]:
u_course_progress.groupby("user_id")

,id,user_id,course_id,modules_achieved,advanced_modules_done,time_created,time_updated,modules_closed,modules_done,course_done,progress_current,progress_current_ratio
0,2020,100120050111530528,3,0,0,NaN,2019-06-05 16:45:32.140810,0,0,False,0.0,0.000000
1,43519,100120050110043028,3,0,0,2019-05-06 19:22:12.861890,2019-06-05 16:45:32.140810,0,0,False,5.0,0.054348
2,26692,100120030111070018,3,0,0,2019-04-15 11:30:32.944271,2019-06-05 16:45:32.140810,0,0,False,0.0,0.000000
3,787,100120060114846069,3,0,0,NaN,2019-06-05 16:45:32.140810,0,0,False,0.0,0.000000
4,2681,100120060110982009,3,8,2,2019-03-22 15:42:21.255839,2019-06-14 11:48:47.039643,1,2,False,74.0,0.804348
...,...,...,...,...,...,...,...,...,...,...,...,...
26179,202673,100119790111022302,237,0,0,2020-09-16 02:25:52.295517,NaN,0,0,False,0.0,0.000000
26180,202230,100120060110010662,237,0,0,2020-09-15 04:10:34.778827,NaN,0,0,False,0.0,0.000000
26181,200338,100119750110038578,237,0,0,2020-09-11 12:29:43.569905,NaN,0,0,False,0.0,0.000000
26182,199512,100120040111161458,237,0,0,2020-09-10 09:04:36.023425,NaN,0,0,False,0.0,0.000000


In [129]:
u_course_progress = pd.read_csv("user_course_progress_done.csv") 

In [35]:
u_course_progress["achived"] = u_course_progress.course_id.map(d) == u_course_progress.modules_achieved

False    23758
True      2426
Name: achived, dtype: int64

In [16]:
u_module_progress.head()

,id,user_id,course_id,course_module_id,progress_current,progress_failed,steps_done,is_achieved,is_closed,course_progress_id,time_created,time_updated,time_closed,time_achieved,time_unlocked,tasks_done,time_done,is_done,achieve_reason
0,503562,100120050110012222,3,52,0.0,0.0,0,False,False,30298,2019-05-01 16:39:05.615147,NaN,NaN,NaN,NaN,0,NaN,False,NaN
1,46805,100120050111530528,3,59,0.0,0.0,0,False,False,2020,NaN,NaN,NaN,NaN,NaN,0,NaN,False,NaN
2,46803,100120050111530528,3,58,0.0,0.0,0,False,False,2020,NaN,NaN,NaN,NaN,NaN,0,NaN,False,NaN
3,46798,100120050111530528,3,57,0.0,0.0,0,False,False,2020,NaN,NaN,NaN,NaN,NaN,0,NaN,False,NaN
4,482893,100120050110594123,3,52,9.0,0.0,20,True,True,29023,2019-04-28 05:34:50.545702,2019-05-01 13:21:08.149917,2019-05-01 13:21:08.149917,2019-05-01 13:13:52.193671,2019-04-30 01:50:19.883097,9,2019-05-01 13:20:23.467270,True,NaN
